Decode hex dumps
================

This workbook manipulates the 95320 dumps collected over serial from the
main.cpp Arduino device.
There are expectations in the next cell for filenames and content.

In [1]:
def load_dump(name: str):
    image = []
    fname = f"{name}.hexdump"
    print(f"Processing {fname}")
    with open(fname) as f:
        for linelf in f:
            line = linelf.rstrip()
            if line.startswith(">"):
                continue
            if line == "read 4KB":
                image.append(b"")
                addr = 0
                continue
            if line.startswith("WIP"):
                print(f"{len(image)} {line}")
                continue

            addr_str, rest = line.split(":")
            data_str = rest.strip().split(" ")
            data = [int(s, base=16) for s in data_str]
            if int(addr_str, base=16) != addr:
                raise RuntimeError(f"addr mismatch {addr_str=} {addr=}")
            addr += len(data)
            image[-1] += bytes(data)
    for i in range(1, len(image)):
        if image[0] != image[i]:
            a, b = image[0], image[i]
            for j in range(min(len(a), len(b))):
                if a[j] != b[j]:
                    print(f"differs at {j}: {a[j]:02x} != {b[j]:02x}")
            raise RuntimeError(
                f"not matching: 0 != {i} : len {len(image[0])} {len(image[i])}"
            )
    with open(f"{name}.bin", "wb") as f:
        f.write(image[0])
    return image[0]


original = load_dump("original")
donor = load_dump("donor")
patched = load_dump("patched")

Processing original.hexdump
1 WIP=0,WEL=0,BP0=0,BP1=0,SRWD=0
2 WIP=0,WEL=0,BP0=0,BP1=0,SRWD=0
3 WIP=0,WEL=0,BP0=0,BP1=0,SRWD=0
Processing donor.hexdump
1 WIP=0,WEL=0,BP0=0,BP1=0,S
2 WIP=0,WEL=0,BP0=0,BP1=0,S
3 WIP=0,WEL=0,BP0=0,BP1=0,S
Processing patched.hexdump
1 WIP=0,WEL=0,BP0=0,BP1=0,SRWD=0
2 WIP=0,WEL=0,BP0=0,BP1=0,SRWD=0
3 WIP=0,WEL=0,BP0=0,BP1=0,SRWD=0


In [2]:
diff = bytes([o ^ d for o, d in zip(original, patched)])
with open("diff.bin", "wb") as f:
    f.write(diff)
sum(1 for d in diff if d != 0)

3

In [3]:
def to_header(name: str, data: bytes):
    with open(name, "w") as f:

        def out(s: str):
            print(s, file=f, end="")

        out(f"unsigned char const {name.replace('.', '_')}[] = {{")
        col = 0
        for d in data:
            if col % 16 == 0:
                out("\n\t")
            out(f"0x{d:02x}, ")
            col += 1
        out(f"\n}};")

In [5]:
to_header("original.hpp", original)
to_header("patched.hpp", bytes([0, 0, 0]) + original[3:])
to_header("donor.hpp", donor)